# ** Diabetes Management with Multiple Devices**

In [ ]:
import DiabetesMonitoring as dm
import holoviews as hv

First things first, enter your information.  We'll use your a1c target to see how your blood sugar is tracking later.

** Update with information about yourself (or whos data you are analyzing).**

In [ ]:
Me = dm.Diabetic(first_name='Mike', 
    sex='M',
    dob=None,
    a1c_target=6.5,
    diagnosis_year=2002,
    diagnosis_type=1)

From, this we can calculate a blood glucose (mg/dl) target:

In [ ]:
print('Daily glucose target is %s'%Me.eag_target)
glucose_target=Me.eag_target

*There are two possible data sources currently supported, one for insulin and the other for continuous blood glucose.*

First, we will identify the location and file name of our Contious Glucose Monitor system (**Dexcom**).

Update the **file_name** in the cell below.

In [ ]:
dexcom = dm.Dexcom(file_folder="DiabetesManagement/Data/Dexcom",
                        file_name="CLARITY_Export__Name_2017-12-15_014001.csv",
                        data_source="Dexcom")

df_d = dexcom.read_data()
glucose = dm.dexcom_clean(df_d)

Then, we will do the same for the insulin delivery information (**Omnipod log**).

Update the **file_name** in the cell below.

In [ ]:
omnipod = dm.Omnipod(file_folder="DiabetesManagement/Data/Omnipod",
                        file_name="Omnipod_20171214.xlsx", 
                        data_source="Omnipod")

df_o = omnipod.read_data()
insulin = dm.omnipod_to_tabular(df_o)
#(isinstance(omnipod,dm.DiabetesData)==True) == (issubclass(dm.Omnipod,dm.DiabetesData)==True)

# ** Glucose and Bolus Overlay **


Combine Omnipod and Dexcom data for an interactive visualization.

In [ ]:
print("Dexcom data ranges from   "+str(min(glucose['event_time']))+"  to  "+str(max(glucose['event_time']))+"\n"
      "\nOmnipod data ranges from  "+str(min(df_o['Date']))+"  to  "+str(max(df_o['Date'])))



You can optionally add *min_date* and *max_date* from the data date ranges to look at more specific time periods.

In [ ]:
import datetime
glucose_chart = dm.glucose_bolus_df(insulin,glucose,
                                    min_date=datetime.datetime(2017,10,1,0,0,0))

In [ ]:
%%opts Overlay [width=900 legend_position='right'] Curve (muted_alpha=0.5 muted_color='black')
%%opts Scatter [width=800 height=400 size_index='growth'] (color=Palette('Category20') size=5)

x = glucose_chart['event_time']
y = glucose_chart['Glucose Value (mg/dL)']
z = glucose_chart['BolusFlg']
t = glucose_target
hv.Curve((x, y), 'Date', 'Blood Sugar', label='CGM') *\
hv.Scatter((x, z), 'Date', 'Bolus', label='Bolus') *\
hv.Curve((x, t), 'Date', 'Target', label='Target')

#### *Use the tools to the right of the chart to explore the data!*

## Review Blood Sugar Tendencies after Bolus

Seeing bg "at bolus" (when eating) and two hours after eating, shows whether Bolus are being used effectively to regular blood sugar.

Recall, you data date ranges from above. You can optionally add *min_date* and *max_date* to the function for a more specific view.  As a default, this will show all values in the datasource.

In [ ]:
combined = dm.bolus_efficacy(insulin, glucose, shift_minutes=120)

In [ ]:
%%opts BoxWhisker [width=400 height=400 xrotation=30 show_title=True, tools=['hover']] 
%%opts BoxWhisker (box_fill_color=Palette('Category10')) norm{-axiswise}

Bolus_dex = hv.Dataset(combined, kdims=['Bolus Time of Day'])
Bolus_dex.to(hv.BoxWhisker, 'Bolus Time of Day', 'Glucose at Bolus') + \
Bolus_dex.to(hv.BoxWhisker, 'Bolus Time of Day', 'Glucose after time period')


## Bolus Use

##### Average Bolus by Type

In [ ]:
x = dm.average_bolus(insulin)[0:1]
x

#### View Daily Total Bolus Use

In [ ]:
%%opts Curve [width=800 height=400 ] (color='green' line_width=2)
daily_bolus = dm.daily_bolus(insulin)
x = daily_bolus['Date']
y = daily_bolus['Total Bolus']
hv.Curve((x, y), 'Date', 'Total Bolus')